In [1]:
pip install yake



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install happytransformer

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/527.3 kB ? eta -:--:--
   ---------------------------------------- 527.3/527.3 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   --------------------------------------- 992.0/992.0 kB 11.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.5 MB ? eta -:--:--
   -------- ------------------------------- 4.2/19.5 MB 19.4 MB/s eta 0:00:01
   ----------------- ---------------------- 8.4/19.5 MB 20.0 MB/s eta 0:00:01
   ------------------------ --------------- 12.1/19.5 MB 18.9 MB/s eta 0:00:01
   -------------------------------- ------- 15.7/19.5 MB 18.3 MB/s eta 0:00:01
   ---------------------------------------  19.4/19.5 MB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 19.5/19.5 MB 17.6 MB/s eta 0:00:00
   ----


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install contractions


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install keybert

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install clean-text



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171057 sha256=7691679179fe750d85b5144832e642ee469b85b4b2e5c094b1ecdd95e0c0a0ce
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\e0\8c\e0\294d2e4ea0e55792bfc99b6b263e4a0511443da7b69af67688
Successfully built emoji



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#import subprocess
#subprocess.run(['python', '-m', 'spacy', 'download', 'en_core_web_trf'])
import pandas as pd
import re
import nltk
import math
import pickle
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import wordnet
import yake
#import FinalScoringDNN
#import generategraphs
from keybert import KeyBERT
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
import numpy as np
import torch
import spacy
from numpy.linalg import norm
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from sentence_transformers import SentenceTransformer
from happytransformer import HappyTextToText, TTSettings
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import sklearn.metrics
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras import initializers
import matplotlib.pyplot as plt


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
dataset = pd.read_excel("FinalDatasetQA.xlsx")
keywordTestData = pd.read_csv("stsb-en-data.csv", names = ['sentence1', 'sentence2', 'keyword_similarity_score'])

In [3]:
from cleantext import clean



Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [4]:
import contractions

def expand_contractions(text):
    # Expands contractions in the text
    return contractions.fix(text)




In [5]:
def text_lower(text):
    return text.lower()


dataset['Model Answer'] = dataset['Model Answer'].apply(text_lower)
dataset['User Answer'] = dataset['User Answer'].apply(text_lower)
dataset['Model Answer'] = dataset['Model Answer'].apply(expand_contractions)
dataset['User Answer'] = dataset['User Answer'].apply(expand_contractions)

#not sure if the last to lines replaces or not

In [6]:
from cleantext import clean

def preprocess(text):
    return clean(
        text,
        lower=True,         # Convert to lowercase
        no_punct=True,      # Remove punctuation

    )
def remove_extra(text):
    return " ".join(text.strip().split())

In [7]:
dataset['Model Answer'] = dataset['Model Answer'].apply(preprocess)
dataset['User Answer'] = dataset['User Answer'].apply(preprocess)
dataset['Model Answer'] = dataset['Model Answer'].apply(remove_extra)
dataset['User Answer'] = dataset['User Answer'].apply(remove_extra)

In [8]:
def normalize(x):
    return x/5
def yake_keywordExtractor(text):
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    deduplication_algo = "seqm"
    windowSize = 2
    numOfKeywords = 101

    # Finetuning the yake model to perform the intended operations
    # Generates the extractor as per hypertuned parameters
    custom_kw_extractor = yake.KeywordExtractor(lan = language,
                                            n = max_ngram_size,
                                            dedupLim = deduplication_threshold,
                                            dedupFunc = deduplication_algo,
                                            windowsSize = windowSize,
                                            top = numOfKeywords,
                                            features = None)

    return custom_kw_extractor.extract_keywords(text)

# Loading the KeyBERT models for comparisions
from keybert import KeyBERT

# Initialize KeyBERT model once
kw_model = KeyBERT()

# Define the keyword extractor function
def keybert_keywordExtractor(text):
    return kw_model.extract_keywords(text)





In [9]:
def keyword_scoring(keywords_text1, keywords_text2):
    match = 0
    total = 0
    synonym_dict = [] # List to include synonym matching

    for token in keywords_text1:
        total += token[1] # Total number of keywords in model answer
    if total == 0:
        return 10 # No keywords in model answer
    # Generating Synonyms for second text
    for var in keywords_text2:
        syn = wordnet.synsets(var[0]) # Synonym Dictionary
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(var[0])
        syn_words = list(set(syn_words))
        temp = []
        wt = word_tokenize(var[0])
        pos = pos_tag(wt)[0][1]
        for i in range(0, len(syn_words)):
            checker_wt = word_tokenize(syn_words[i])
            checker_pos = pos_tag(wt)[0][1]
            if pos == checker_pos:
                temp.append(syn_words[i])
        synonym_dict = synonym_dict + temp # Enhancing the synonym dict for direct match compatibility

    # Calculating the total number of matching keywords
    for token in keywords_text1:
        syn = wordnet.synsets(token[0])
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(token[0])
        syn_words = list(set(syn_words))
        if len(set(syn_words).intersection(set(synonym_dict))) != 0:
            match += token[1]
    # Keyword score is number of matching keywords over total number of model keywords (normalized to 10)
    return match * 10 / total

In [10]:
print("hello i am working ")

hello i am working 


In [11]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [15]:
# Similarity Scoring
# 3 PAIRS  OF KAPDE 
#CHAR UNDERWEAR
# ODNE + BICHNANE
#JACKET 
def cos_sim(sent1_emb, sent2_emb):
    cos = np.dot(sent1_emb, sent2_emb)/(norm(sent1_emb)*norm(sent2_emb))
    return cos
mod_ans = dataset.iloc[:, 2].values # List of Model Answers
usr_ans = dataset.iloc[:, 3].values # List of User Answers
def similarity_scoring(text1, text2, model):
    mod_sent = sent_tokenize(text1) # Individual sentences in model answer
    usr_sent = sent_tokenize(text2) # Individual sentences in user answer
    mod_emb = [] # Incorporating Model sentences embeddings
    usr_emb = [] # Incorporating User sentences embeddings
    for sent in mod_sent:
        sent_emb = model.encode(sent)
        mod_emb.append(sent_emb)
    for sent in usr_sent:
        sent_emb = model.encode(sent)
        usr_emb.append(sent_emb)
    n = len(mod_sent)
    m = len(usr_sent)
    match = 0
    for i in range (0, n):
        for j in range (0, m):
            if cos_sim(mod_emb[i], usr_emb[j]) >= 0.75:
                # Defining cosine threshold at 0.75
                match += 1
                break
    return match / n

# Adding Summarizer
# creating a t5 summarizer model
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Can replace to gpu for faster processing

def summarizer(text):
    preprocessed_text = text.strip().replace('\n', '')
    t5_input_text = 'summarize: ' + preprocessed_text
    tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)
    summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Adding Grammar Corrector
def grammar_corrector(text):
    happy_tt = HappyTextToText("T5", "prithivida/grammar_error_correcter_v1") # Loading the corrector library
    text = "gec: " + text # Necessary pre-processing
    settings = TTSettings(do_sample=True, top_k=10, temperature=0.5, min_length=1, max_length=100)
    result = happy_tt.generate_text(text, args=settings)
    return result.text

# simcse = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large') # SimCSE Model
sbert = SentenceTransformer('stsb-mpnet-base-v2')
# hf = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

num_of_inputs = len(mod_ans) # Total number of Answers
# sim_score = [0] * num_of_inputs # List to keep respective similarity score
sbert_score = [0] * num_of_inputs
# hf_score = [0] * num_of_inputs

# Similarity matrix in use for the calculation
for i in range(0, num_of_inputs):
    curr_mod = mod_ans[i]
    curr_usr = usr_ans[i]
    cur_usr = grammar_corrector(curr_usr) # grammar correction on user answer
    curr_usr += summarizer(curr_usr) # adding summary of user answer to user answer
   
    sbert_val = similarity_scoring(curr_mod, curr_usr, sbert)
    sbert_score[i] = sbert_val * 10
   
#Converting list to numpy array

sbert_score = np.array(sbert_score)


print("Similarity/Semantic scores are generated Successfully!!!")


        

# Addition of the fields to calculate the yake scores
dataset['yake_keys_sentence1'] = dataset['Model Answer'].apply(yake_keywordExtractor)
dataset['yake_keys_sentence2'] = dataset['User Answer'].apply(yake_keywordExtractor)



# Adding the Keyword Score in dataset
dataset['yake_score'] = dataset[['yake_keys_sentence1', 'yake_keys_sentence2']].apply(lambda x : keyword_scoring(x.yake_keys_sentence1,
                                                                                        x.yake_keys_sentence2),
                                                                                        axis = 1)


print("Keyword scores are generated Successfully!!!")
# Dropping the additional, not functional fields
dataset.drop(['yake_keys_sentence1', 'yake_keys_sentence2'], axis = 1, inplace = True)



12/27/2024 13:20:50 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
12/27/2024 13:20:50 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
12/27/2024 13:20:52 - INFO - happytransformer.happy_transformer -   Initializing a pipeline
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

12/27/2024 13:48:34 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
12/27/2024 13:48:34 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
12/27/2024 13:48:35 - INFO - happytransformer.happy_transformer -   Initializing a pipeline
Device set to use cuda:0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity/Semantic scores are generated Successfully!!!
Keyword scores are generated Successfully!!!


In [18]:
dataset['sbert_score'] = sbert_score.tolist()

In [19]:
dataset.head()

,S. No,Question,Model Answer,User Answer,Actual Score (0-10),yake_score,sbert_score
0,1,What is an Operating Systems?Why do we need an...,an operating system os is a collection of soft...,an operating system is a program that acts as ...,4.0,6.580460,10.0
1,2,Explain different types of tasks done by OS an...,operating system services and facilites can be...,following are the services provided by an oper...,4.0,5.133728,10.0
2,3,Give the view of OS as a Resource Manager,the concept of an operating system as providin...,from a highlevel perspective an operating syst...,6.0,6.394426,10.0
3,4,Give the view of OS as an extended machine.,the operating systems perform two basically un...,from another perspective an operating system o...,4.0,4.514964,0.0
4,5,What is Deadlock Avoidance?,deadlock avoidance is a set of techniques used...,deadlock avoidance is a technique used in oper...,3.0,6.394955,0.0


In [20]:
import math
import warnings
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential, initializers
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Replace `dataset` with your actual DataFrame variable
df = dataset  # Ensure `dataset` is loaded before running this code

# Round values in a pandas Series to one decimal place
def roundVal(X):
    y = X.tolist()
    for i in range(len(y)):
        y[i] = round(y[i], 1)
    return y

# Define score categories
keyword = [ 'yake_score']
semantic = ['sbert_score']

data = []

for i in range(len(keyword)):
    for j in range(len(semantic)):
        arr = [keyword[i], semantic[j]]
        # Selecting relevant columns
        df1 = df[[keyword[i], semantic[j], 'Actual Score (0-10)']]

        # Rounding score values
        df1[keyword[i]] = roundVal(df1[keyword[i]])
        df1[semantic[j]] = roundVal(df1[semantic[j]])

        warnings.simplefilter(action='ignore')

        X = df1[[keyword[i], semantic[j]]]
        y = df1['Actual Score (0-10)']

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, random_state=9, train_size=0.8, shuffle=True
        )

        # Define the model
        model = Sequential()
        model.add(Dense(units=8, input_dim=2, kernel_initializer='normal', activation='relu'))
        model.add(Dense(units=16, kernel_initializer=initializers.RandomNormal(stddev=1), 
                        bias_initializer=initializers.Zeros(), activation='tanh'))
        model.add(Dense(units=32, kernel_initializer='normal', activation='relu'))
        model.add(Dense(units=64, kernel_initializer=initializers.RandomNormal(stddev=1), 
                        bias_initializer=initializers.Zeros(), activation='tanh'))
        model.add(Dense(1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='adam')

        # Train the model
        model.fit(X_train, y_train, batch_size=6, epochs=100, verbose=0)

        # Save the model and print summary for a specific condition
        if i == 1 and j == 1:
            model.summary()
            trainedMLmodelFilename = '../results/SubjectiveAnswerChecker.sav'
            pickle.dump(model, open(trainedMLmodelFilename, 'wb'))
            print("Model generated Successfully!!!")

        # Calculate RMSE
        rmseTest = math.sqrt(
            mean_squared_error(
                y_test / 10, np.round(model.predict(X_test), 1) / 10
            )
        ) / 5
        rmseTrain = math.sqrt(
            mean_squared_error(
                y_train / 10, np.round(model.predict(X_train), 1) / 10
            )
        ) / 5
        rmseTotal = math.sqrt(
            mean_squared_error(
                y / 10, np.round(model.predict(X), 1) / 10
            )
        ) / 5

        # Append results and print them
        arr.extend([rmseTrain, rmseTest, rmseTotal])
        data.append(arr)
        print(f"Combination: {keyword[i]} & {semantic[j]}")
        print(f"RMSE (Train): {rmseTrain}, RMSE (Test): {rmseTest}, RMSE (Total): {rmseTotal}")
        print("-" * 50)

        # Clear session
        tf.keras.backend.clear_session()

print("All combinations processed successfully.")


C:\Users\Hp\AppData\Local\Temp\ipykernel_13912\604472426.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[keyword[i]] = roundVal(df1[keyword[i]])
C:\Users\Hp\AppData\Local\Temp\ipykernel_13912\604472426.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[semantic[j]] = roundVal(df1[semantic[j]])


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Combination: yake_score & sbert_score
RMSE (Train): 0.04155901045297878, RMSE (Test): 0.04714702625899766, RMSE (Total): 0.04273510848094296
--------------------------------------------------

All combinations processed successfully.
